<a href="https://colab.research.google.com/github/johanjun/MLOps_study/blob/main/wandb_sweeps(practice).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q wandb

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 138 kB 22.5 MB/s 
     |████████████████████████████████| 133 kB 28.4 MB/s 
     |████████████████████████████████| 97 kB 6.4 MB/s 
     |████████████████████████████████| 170 kB 36.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 62 kB 775 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [7]:
# %load train_lib.py

def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = {
        'layer1_size': 128,
        'layer1_activation': 'relu',
        'dropout_rate': 0.2,
        'optimizer': 'adam',
        'learning_rate': 0.01

    }
    wandb.init(project='sweep-practice',
               config=config_defaults,
               magic=True)
    
    config = wandb.config

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images.shape
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(wandb.config.layer1_size, activation=config.layer1_activation),
        tf.keras.layers.Dropout(config.dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    
    if config.optimizer == 'rmsprop':
      opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
    else:
      opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5,
                  validation_data=(test_images, test_labels))


In [9]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'layer1_size': {
            'values': [64, 96, 128]
        },
        'layer1_activation': {
            'values': ['relu', 'sigmoid']
        },
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
        'optimizer': {
            'values': ['adam', 'rmsprop']
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]
        }
    }
}

In [11]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='sweep-practice')

Create sweep with ID: hwjkd12t
Sweep URL: https://wandb.ai/johanjun/sweep-practice/sweeps/hwjkd12t


In [12]:
wandb.agent(sweep_id, function=train)

1805/1875 [===========================>..] - ETA: 0s - loss: 0.3563 - accuracy: 0.8689

wandb: Ctrl + C detected. Stopping sweep.


In [13]:
with open('train.py', 'w') as f:
  f.write("""
import numpy as np
import tensorflow as tf
import wandb
config_defaults = {
    'layer1_size': 128,
    'layer1_activation': 'relu',
    'dropout_rate': 0.2,
    'optimizer': 'adam',
    'learning_rate': 0.01

}
wandb.init(project='sweep-practice',
            config=config_defaults,
            magic=True)

config = wandb.config

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images.shape
train_images = train_images / 255.0
test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(wandb.config.layer1_size, activation=config.layer1_activation),
    tf.keras.layers.Dropout(config.dropout_rate),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])


if config.optimizer == 'rmsprop':
  opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
else:
  opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5,
              validation_data=(test_images, test_labels))

  """)

In [14]:
!head train.py


import numpy as np
import tensorflow as tf
import wandb
config_defaults = {
    'layer1_size': 128,
    'layer1_activation': 'relu',
    'dropout_rate': 0.2,
    'optimizer': 'adam',
    'learning_rate': 0.01


In [15]:
!wandb agent johanjun/sweep-practice/di6d6t7o

wandb: Starting wandb agent 🕵️
2021-08-03 07:46:47,576 - wandb.wandb_agent - INFO - Running runs: []
2021-08-03 07:46:47,901 - wandb.wandb_agent - INFO - Agent received command: run
2021-08-03 07:46:47,901 - wandb.wandb_agent - INFO - Agent starting run with config:
	dropout_rate: 0.112387613418287
	layer1_activation: relu
	layer1_size: 94
	learning_rate: 0.05750846514328967
	optimizer: adam
2021-08-03 07:46:47,903 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train.py --dropout_rate=0.112387613418287 --layer1_activation=relu --layer1_size=94 --learning_rate=0.05750846514328967 --optimizer=adam
2021-08-03 07:46:48.359948: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
wandb: Currently logged in as: johanjun (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep
2021-08-03 07:46:52.405140: I tensorflow/stream_executor/platform/defau

KeyboardInterrupt: ignored